In [2]:
import matplotlib
import pandas as pd


In [32]:
data=pd.read_csv('./Disease_symptom_and_patient_profile_dataset.csv',header=0)
data=data.dropna()
data.columns=[column.replace(' ','_') for column in data.columns ]

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Disease               349 non-null    object
 1   Fever                 349 non-null    object
 2   Cough                 349 non-null    object
 3   Fatigue               349 non-null    object
 4   Difficulty_Breathing  349 non-null    object
 5   Age                   349 non-null    int64 
 6   Gender                349 non-null    object
 7   Blood_Pressure        349 non-null    object
 8   Cholesterol_Level     349 non-null    object
 9   Outcome_Variable      349 non-null    object
dtypes: int64(1), object(9)
memory usage: 27.4+ KB


In [34]:
data.head()

,Disease,Fever,Cough,Fatigue,Difficulty_Breathing,Age,Gender,Blood_Pressure,Cholesterol_Level,Outcome_Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive


In [35]:
# one-hot
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for column in data.select_dtypes(include=['object']).columns:
    data[column]=label_encoder.fit_transform(data[column])


In [36]:
x=data.drop('Outcome_Variable',axis=1)
y=data['Outcome_Variable']


In [37]:
display(x,y)

,Disease,Fever,Cough,Fatigue,Difficulty_Breathing,Age,Gender,Blood_Pressure,Cholesterol_Level
0,56,1,0,1,1,19,0,1,2
1,24,0,1,1,0,25,0,2,2
2,37,0,1,1,0,25,0,2,2
3,6,1,1,0,1,25,1,2,2
4,6,1,1,0,1,25,1,2,2
...,...,...,...,...,...,...,...,...,...
344,101,1,0,1,0,80,0,0,0
345,101,1,0,1,0,85,1,0,0
346,101,1,0,1,0,85,1,0,0
347,101,1,0,1,0,90,0,0,0


0      1
1      0
2      0
3      1
4      1
      ..
344    1
345    1
346    1
347    1
348    1
Name: Outcome_Variable, Length: 349, dtype: int64

In [52]:
import numpy as np
from scipy import stats
class kNN():
    '''k-Nearest Neighbours'''
    # Initialise
    def __init__(self, k=3, metric='euclidean', p=None):
        self.k = k
        self.metric = metric
        self.p = p
    
    # Euclidean distance (l2 norm)
    def euclidean(self, v1, v2):
        return np.sqrt(np.sum((v1-v2)**2))
    
    # Manhattan distance (l1 norm)
    def manhattan(self, v1, v2):
        return np.sum(np.abs(v1-v2))
    
    # Minkowski distance (lp norm)
    def minkowski(self, v1, v2, p=2):
        return np.sum(np.abs(v1-v2)**p)**(1/p)
        
    # Store train set
    def fit(self, X_train, y_train):
        self.X_train = X_train.to_numpy()
        self.y_train = y_train.to_numpy()
        
    # Make predictions
    def predict(self, X_test):
        preds = []
        X_test=X_test.to_numpy()
        # Loop over rows in test set
        for test_row in X_test:
            nearest_neighbours = self.get_neighbours(test_row)
            majority = stats.mode(nearest_neighbours)[0][0]
            preds.append(majority)
        return np.array(preds)
    
    # Get nearest neighbours
    def get_neighbours(self, test_row):
        distances = list()
        
        # Calculate distance to all points in X_train
        for (train_row, train_class) in zip(self.X_train, self.y_train):
            print(train_row,test_row)
            if self.metric=='euclidean':
                dist = self.euclidean(train_row, test_row)
            elif self.metric=='manhattan':
                dist = self.manhattan(train_row, test_row)
            elif self.metric=='minkowski':
                dist = self.minkowski(train_row, test_row, self.p)
            else:
                raise NameError('Supported metrics are euclidean, manhattan and minkowski')
            distances.append((dist, train_class))
            
        # Sort distances
        distances.sort(key=lambda x: x[0])
        
        # Identify k nearest neighbours
        neighbours = list()
        for i in range(self.k):
            neighbours.append(distances[i][1])
            
        return neighbours

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)
k=data['Disease'].unique().size

sklearnClassifier =KNeighborsClassifier(k)
sklearnClassifier.fit(x_train,y_train)
y_predict=sklearnClassifier.predict(x_test)
score=accuracy_score(y_test,y_predict)


In [53]:

myclassifier =kNN(k)
myclassifier.fit(x_train,y_train)
y_predict=myclassifier.predict(x_test)


[90  1  0  0  0 40  1  0  2] [55  1  1  1  0 42  0  0  0]
[90  1  0  0  0 40  1  0  2] [55  1  1  1  0 42  0  0  0]
[31  0  0  1  0 35  0  2  0] [55  1  1  1  0 42  0  0  0]
[31  0  0  1  0 35  0  2  0] [55  1  1  1  0 42  0  0  0]
[32  1  0  0  0 40  0  0  0] [55  1  1  1  0 42  0  0  0]
[32  1  0  0  0 40  0  0  0] [55  1  1  1  0 42  0  0  0]
[31  1  1  1  1 29  1  0  2] [55  1  1  1  0 42  0  0  0]
[31  1  1  1  1 29  1  0  2] [55  1  1  1  0 42  0  0  0]
[112   1   1   0   1  50   1   0   2] [55  1  1  1  0 42  0  0  0]
[112   1   1   0   1  50   1   0   2] [55  1  1  1  0 42  0  0  0]
[77  0  1  0  0 38  0  2  0] [55  1  1  1  0 42  0  0  0]
[77  0  1  0  0 38  0  2  0] [55  1  1  1  0 42  0  0  0]
[110   0   1   0   0  55   1   2   1] [55  1  1  1  0 42  0  0  0]
[110   0   1   0   0  55   1   2   1] [55  1  1  1  0 42  0  0  0]
[98  0  1  1  0 30  1  2  2] [55  1  1  1  0 42  0  0  0]
[98  0  1  1  0 30  1  2  2] [55  1  1  1  0 42  0  0  0]
[77  0  0  1  0 55  1  0  2] [55  1 

/tmp/ipykernel_1021793/3534598684.py:36: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  majority = stats.mode(nearest_neighbours)[0][0]


[90  1  0  0  0 40  1  0  2] [12  0  0  1  0 60  1  0  0]
[90  1  0  0  0 40  1  0  2] [12  0  0  1  0 60  1  0  0]
[31  0  0  1  0 35  0  2  0] [12  0  0  1  0 60  1  0  0]
[31  0  0  1  0 35  0  2  0] [12  0  0  1  0 60  1  0  0]
[32  1  0  0  0 40  0  0  0] [12  0  0  1  0 60  1  0  0]
[32  1  0  0  0 40  0  0  0] [12  0  0  1  0 60  1  0  0]
[31  1  1  1  1 29  1  0  2] [12  0  0  1  0 60  1  0  0]
[31  1  1  1  1 29  1  0  2] [12  0  0  1  0 60  1  0  0]
[112   1   1   0   1  50   1   0   2] [12  0  0  1  0 60  1  0  0]
[112   1   1   0   1  50   1   0   2] [12  0  0  1  0 60  1  0  0]
[77  0  1  0  0 38  0  2  0] [12  0  0  1  0 60  1  0  0]
[77  0  1  0  0 38  0  2  0] [12  0  0  1  0 60  1  0  0]
[110   0   1   0   0  55   1   2   1] [12  0  0  1  0 60  1  0  0]
[110   0   1   0   0  55   1   2   1] [12  0  0  1  0 60  1  0  0]
[98  0  1  1  0 30  1  2  2] [12  0  0  1  0 60  1  0  0]
[98  0  1  1  0 30  1  2  2] [12  0  0  1  0 60  1  0  0]
[77  0  0  1  0 55  1  0  2] [12  0 

In [ ]:
from sklearn.model_selection import train_test_split
X=data.drop('')

x_train, x_test, y_train, y_test=train_test_split()



In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

fpr, tpr, thresholds = roc_curve(y_true, y_scores)
auc = roc_auc_score(y_true, y_scores)